<a href="https://colab.research.google.com/github/tresky1977/TRY2/blob/main/Project_Python_for_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing libraries and authenticate

In [64]:
!pip install gspread google-auth pandas

import gspread
from google.auth import default
import pandas as pd

creds, _ = default()
client = gspread.authorize(creds)

Importing Google Sheets 'Slovakia demography' with two sheets 'Demography' and 'Gender'.

In [65]:
sheet_url = "https://docs.google.com/spreadsheets/d/18ce5TG-c-Zz3uNVQVhPLZPT_jsdV-cRbo3rhSbB2qsg/edit?usp=sharing"
sheet = client.open_by_url(sheet_url)

worksheet_1 = sheet.worksheet('Demography')
worksheet_2 = sheet.worksheet('Gender')

Reading data

In [66]:
data_1 = worksheet_1.get_all_values()
data_2 = worksheet_2.get_all_values()

Converting data to Dataframe using Pandas libraries

In [67]:
df_1 = pd.DataFrame(data_1[1:], columns=data_1[0])
df_2 = pd.DataFrame(data_2[1:], columns=data_2[0])

Normalizing the column names by stripping any leading/trailing whitespace and converting to **lowercase**

In [68]:
df_1.columns = df_1.columns.str.strip().str.lower()
df_2.columns = df_2.columns.str.strip().str.lower()

Displaying 'heads' of sheets to verify succesfull import

In [69]:
print("Demography Data:")
print(df_1.head())

print("\nGender Data:")
print(df_2.head())

Demography Data:
   year population avg population due 31.12 total growth natural increase  \
0  1993      5,349,108            5,361,116       22,300           20,549   
1  1994      5,347,413            5,356,207       19,752           14,984   
2  1995      5,363,638            5,367,790       11,583            8,741   
3  1996      5,373,810            5,378,932       11,142            8,887   
4  1997      5,383,214            5,387,650        8,718            6,987   

     born    died migration arrived   left  
0  73,256  52,707     1,751   9,106  7,355  
1  66,370  51,386     4,768   4,922    154  
2  61,427  52,686     2,842   3,055    213  
3  60,123  51,236     2,255   2,477    222  
4  59,111  52,124     1,731   2,303    572  

Gender Data:
   year population avg population due 31.12      women        men
0  1993      5,349,108            5,361,116  2,748,945  2,612,171
1  1994      5,347,413            5,356,207  2,747,306  2,608,901
2  1995      5,363,638            5,36

Ensuring that the column 'year' is int

In [70]:
df_1['year'] = df_1['year'].astype(int)
df_2['year'] = df_2['year'].astype(int)

In this step 'Demography' and 'Gerder' sheets are joined on 'Year' column
- merging Dataframes using inner join
- display new DF


In [71]:
merged_df = pd.merge(df_1, df_2, on='year', how='inner')
merged_df.head()

,year,population avg_x,population due 31.12_x,total growth,natural increase,born,died,migration,arrived,left,population avg_y,population due 31.12_y,women,men
0,1993,"5,349,108","5,361,116","22,300","20,549","73,256","52,707","1,751","9,106","7,355","5,349,108","5,361,116","2,748,945","2,612,171"
1,1994,"5,347,413","5,356,207","19,752","14,984","66,370","51,386","4,768","4,922",154,"5,347,413","5,356,207","2,747,306","2,608,901"
2,1995,"5,363,638","5,367,790","11,583","8,741","61,427","52,686","2,842","3,055",213,"5,363,638","5,367,790","2,754,078","2,613,712"
3,1996,"5,373,810","5,378,932","11,142","8,887","60,123","51,236","2,255","2,477",222,"5,373,810","5,378,932","2,760,498","2,618,434"
4,1997,"5,383,214","5,387,650","8,718","6,987","59,111","52,124","1,731","2,303",572,"5,383,214","5,387,650","2,765,645","2,622,005"


Deleting duplicated columns and unnecessary columns

In [72]:
columns_to_drop = ['population due 31.12_y', 'population avg_y', 'population avg_x']
merged_df = merged_df.drop(columns=columns_to_drop)
merged_df.head()

,year,population due 31.12_x,total growth,natural increase,born,died,migration,arrived,left,women,men
0,1993,"5,361,116","22,300","20,549","73,256","52,707","1,751","9,106","7,355","2,748,945","2,612,171"
1,1994,"5,356,207","19,752","14,984","66,370","51,386","4,768","4,922",154,"2,747,306","2,608,901"
2,1995,"5,367,790","11,583","8,741","61,427","52,686","2,842","3,055",213,"2,754,078","2,613,712"
3,1996,"5,378,932","11,142","8,887","60,123","51,236","2,255","2,477",222,"2,760,498","2,618,434"
4,1997,"5,387,650","8,718","6,987","59,111","52,124","1,731","2,303",572,"2,765,645","2,622,005"


Since I has trouble with certain manipulation I asked for data types

In [73]:
print(merged_df.dtypes)


year                       int64
population due 31.12_x    object
total growth              object
natural increase          object
born                      object
died                      object
migration                 object
arrived                   object
left                      object
women                     object
men                       object
dtype: object


Converting all variables to int64

In [74]:

for col in merged_df.columns:
    merged_df[col] = merged_df[col].astype(str).str.replace(',', '').replace('nan', '0').astype('int64')

print(merged_df.dtypes)
merged_df.head()


year                      int64
population due 31.12_x    int64
total growth              int64
natural increase          int64
born                      int64
died                      int64
migration                 int64
arrived                   int64
left                      int64
women                     int64
men                       int64
dtype: object


,year,population due 31.12_x,total growth,natural increase,born,died,migration,arrived,left,women,men
0,1993,5361116,22300,20549,73256,52707,1751,9106,7355,2748945,2612171
1,1994,5356207,19752,14984,66370,51386,4768,4922,154,2747306,2608901
2,1995,5367790,11583,8741,61427,52686,2842,3055,213,2754078,2613712
3,1996,5378932,11142,8887,60123,51236,2255,2477,222,2760498,2618434
4,1997,5387650,8718,6987,59111,52124,1731,2303,572,2765645,2622005


There have been few columns for total population usind different methodology. I need to make sure that men and women are visualised properly within population total.

In [75]:
merged_df['population verification'] = merged_df['women'] + merged_df['men']
merged_df.head()

,year,population due 31.12_x,total growth,natural increase,born,died,migration,arrived,left,women,men,population verification
0,1993,5361116,22300,20549,73256,52707,1751,9106,7355,2748945,2612171,5361116
1,1994,5356207,19752,14984,66370,51386,4768,4922,154,2747306,2608901,5356207
2,1995,5367790,11583,8741,61427,52686,2842,3055,213,2754078,2613712,5367790
3,1996,5378932,11142,8887,60123,51236,2255,2477,222,2760498,2618434,5378932
4,1997,5387650,8718,6987,59111,52124,1731,2303,572,2765645,2622005,5387650


I decided that I do not need years 1993 and 1994

In [76]:
filtered_df = merged_df[(merged_df['year'] >= 1995) & (merged_df['year'] <= 2023)]
filtered_df.head()


,year,population due 31.12_x,total growth,natural increase,born,died,migration,arrived,left,women,men,population verification
2,1995,5367790,11583,8741,61427,52686,2842,3055,213,2754078,2613712,5367790
3,1996,5378932,11142,8887,60123,51236,2255,2477,222,2760498,2618434,5378932
4,1997,5387650,8718,6987,59111,52124,1731,2303,572,2765645,2622005,5387650
5,1998,5393382,5732,4426,57582,53156,1306,2052,746,2769690,2623692,5393382
6,1999,5398657,5275,3821,56223,52402,1454,2072,618,2773531,2625126,5398657


Last step before the visualisation is looking for 'null' in DF

In [77]:
has_nulls = merged_df.isnull().values.any()
print("Any null values in DataFrame:", has_nulls)

Any null values in DataFrame: False


Visualisation part starts by importing plotly, which is my favourite visualisation library.

In [81]:
import plotly.express as px
import plotly.graph_objects as go

In [119]:
# Custom color palette (https://plotly.com/python/discrete-color/)
custom_colors = ['#1f77b4', '#ff7f0e']  # Blue for men, orange for women

# Creating the line plot with custom colors
fig = px.line(merged_df, x='year', y=['men', 'women'], title='Men and Women over Years',
              color_discrete_sequence=custom_colors)

# Updating the layout for better visual appeal
fig.update_layout(
    title={'text': 'Men and Women over Years', 'x': 0.5, 'xanchor': 'center'},
    xaxis_title='Year',
    yaxis_title='Count',
    legend_title='Gender',
    template='plotly_white',
)

# Adding markers for better data point visibility
fig.update_traces(marker=dict(size=6))

# Visualising the plot
fig.show()


In [99]:
fig = go.Figure()

# Adding net migration (difference) data to the plot as a bar chart
fig.add_trace(go.Bar(
    x=merged_df['year'],
    y=merged_df['migration'],
    name='Net Migration (Difference)',
    marker=dict(color='lightgoldenrodyellow'),  #color
    width=0.5  #  bar width
))

# Add migration (arrived) data to the plot as a line chart
fig.add_trace(go.Scatter(
    x=merged_df['year'],
    y=merged_df['arrived'],
    mode='lines+markers',
    name='Migration',
    line=dict(color='skyblue', width=2),
    marker=dict(symbol='square', size=6)
))

# Add emigration (left) data to the plot as a line chart
fig.add_trace(go.Scatter(
    x=merged_df['year'],
    y=merged_df['left'],
    mode='lines+markers',
    name='Emigration',
    line=dict(color='lightcoral', width=2),
    marker=dict(symbol='diamond', size=6)
))

# Updating layout
fig.update_layout(
    title='Migration Development in Slovakia by Years',
    xaxis=dict(title='Year'),
    yaxis=dict(
        title='Number of People',
        tickformat='.2s'  # 's' format for 'k' as thousends
    ),
    width=1000,  # width of the entire chart
    height=600,  # height of the entire chart
    barmode='overlay'  # Overlay the bar chart on the line charts
)

fig.show()

In [136]:
# Create figure
fig = go.Figure()

# Add bar chart for 'born' with new label and friendly color
fig.add_trace(go.Bar(
    x=merged_df['year'],
    y=merged_df['born'],
    name='Natality Total',
    marker_color='lightblue'
))

# Add bar chart for 'died' with new label and friendly color
fig.add_trace(go.Bar(
    x=merged_df['year'],
    y=merged_df['died'],
    name='Mortality Total',
    marker_color='lightcoral'
))

# Add line chart for 'natural increase' with friendly color
fig.add_trace(go.Scatter(
    x=merged_df['year'],
    y=merged_df['natural increase'],
    mode='lines+markers',
    name='Natural Increase',
    line=dict(color='black', width=2),
    marker=dict(color='black', size=8)
))

# Update layout with off-white background, black axis lines, and no grid lines
fig.update_layout(
    title='Mortality vs. Natality between 1993 and 2023',
    xaxis_title='Year',
    yaxis_title='Values',
    barmode='group',
    legend_title='Legend',
    plot_bgcolor='rgb(248, 248, 248)',  # Off-white background for plot area
    paper_bgcolor='rgb(248, 248, 248)',  # Off-white background for entire figure
    xaxis=dict(
        linecolor='black',  # Axis line color
        showgrid=False,  # Hide grid lines
        zerolinecolor='black',  # Zero line color
        zerolinewidth=0.1  # Zero line width
    ),
    yaxis=dict(
        linecolor='black',  # Axis line color
        showgrid=False,  # Hide grid lines
        zerolinecolor='black',  # Zero line color
        zerolinewidth=0.1  # Zero line width
    )
)

# Show the plot
fig.show()
